<H1>Script to Retrieve Data from Call Reports<H1>

<b>Required Libraries:</b><br>
Zeep (Python SOAP client based on lxml / requests)<br>
Beautiful Soap (Python library for pulling data out of HTML and XML files)<br>
Flask (extensible web microframework for building web applications with Python)<br>
Pandas (data structures and data analysis tools for the Python programming)<br>
Requests (Send and request data over HTTP)<br>

In [19]:
from flask import Flask, render_template, flash, request
from wtforms import Form, TextField, TextAreaField, validators, StringField, SubmitField
from datetime import datetime
import pandas as pd
from bs4 import BeautifulSoup
import requests 
from zeep import Client, settings
from zeep.wsse.username import UsernameToken

<b>User Token Required to with Web Services Description Language</b>

In [2]:
response = requests.get("https://cdr.ffiec.gov/public/ManageFacsimiles.aspx")
client = Client('https://cdr.ffiec.gov/Public/PWS/WebServices/RetrievalService.asmx?WSDL', wsse=UsernameToken('ankur_sia', 'JACWhNKORrQ7ZcPOo1oL'))

<b>Function to check response and user access with website</b>

In [3]:
def response_status():
    status_code = response.status_code
    if (status_code==200 and client.service.TestUserAccess()==True):
        return("Response is working and Connection SetUp")
    else:
        return("Error, page not found or Issue with authentication")

<b>Function to get raw report in XBRL format from the web service</b>

In [4]:
def get_report(report_date, ID_RSSD):
    try:
        resp = client.service.RetrieveFacsimile('Call', report_date, 'ID_RSSD', ID_RSSD, 'XBRL')
        xbrl_str = resp.decode('utf-8')
        return xbrl_str
    except:
        return ("Error")
   


In [5]:
# def get_report(report_date, ID_RSSD):
#     resp = client.service.RetrieveFacsimile('Call', report_date, 'ID_RSSD', ID_RSSD, 'XBRL')
#     print(resp)
#     xbrl_str = resp.decode('utf-8')
#     return xbrl_str

In [6]:
#get_report("06-30-2019", 49740)

<b>Function to clean raw report using Beautiful Soup</b>

In [7]:
def report_clean(report_var):
    soup = BeautifulSoup(report_var)
    return soup

<b>Function to get specific information from call reports</b>

In [8]:
def get_specefic_codes(lst, rpt):
    values_store = []
    err = 404
    for i in enumerate(lst):
        out = rpt.find(i)
        if(out==None):
            return err
            break
        temp = out.decode_contents()
        values_store.append(temp)
    return values_store

<b>Requested Codes</b>

In [9]:
# Apple_Bank = 249612
# TD_Bank = 497404
# rep_codes = ["cc:rcfdf159", "cc:rcfd1460", "cc:rcfdf161",
#             "cc:rconf175", "cc:rcon3500", "cc:rconf181",
#              "cc:rconf177", "cc:rcon3501", "cc:rconf183", 
#              "cc:riadc893", "cc:riad3588","cc:riadc897",
#             "cc:riadc894", "cc:riad3589", "cc:riadc898"] 
            
rep_codes =      ["cc:rconf159", "cc:rcon1460", "cc:rconf161", 
                "cc:rconf175", "cc:rcon3500", "cc:rconf181",
              "cc:rconf177", "cc:rcon3501", "cc:rconf183", 
              "cc:riadc893", "cc:riad3588","cc:riadc897",
             "cc:riadc894", "cc:riad3589", "cc:riadc898"]

In [10]:
# report_bank = get_report("06-30-2019", 3918898)
# report_bank_clean = report_clean(report_bank)
# bank_values = get_specefic_codes(rep_codes,report_bank_clean)
# if (bank_values==404):
#     df1 = pd.DataFrame([bank_values], columns=["ERROR WITH REPORT CODES"])
# else:
#     df1 = pd.DataFrame([bank_values], columns=[
#             "cc:rconf175", "cc:rcon3500", "cc:rconf181",
#              "cc:rconf177", "cc:rcon3501", "cc:rconf183", 
#              "cc:riadc893", "cc:riad3588","cc:riadc897",
#             "cc:riadc894", "cc:riad3589", "cc:riadc898"]  )
# print(df1)

In [11]:
def results(date, i):
    report_bank = get_report(date, i )
    report_bank_clean = report_clean(report_bank)
    bank_values = get_specefic_codes(rep_codes,report_bank_clean)
    if (bank_values==404):
        df = pd.DataFrame(["!!!***********!!!"], columns=["ERROR WITH REPORT CODES OR BANK ID OR DATE"])
        return(df)
    else:
#         df = pd.DataFrame([bank_values], columns=["cc:rcfdf159", "cc:rcfd1460", "cc:rcfdf161",
#             "cc:rconf175", "cc:rcon3500", "cc:rconf181",
#              "cc:rconf177", "cc:rcon3501", "cc:rconf183", 
#              "cc:riadc893", "cc:riad3588","cc:riadc897",
#             "cc:riadc894", "cc:riad3589", "cc:riadc898"]  )
        df = pd.DataFrame([bank_values], columns=["cc:rconf159", "cc:rcon1460", "cc:rconf161",
             "cc:rconf175", "cc:rcon3500", "cc:rconf181",
              "cc:rconf177", "cc:rcon3501", "cc:rconf183", 
              "cc:riadc893", "cc:riad3588","cc:riadc897",
             "cc:riadc894", "cc:riad3589", "cc:riadc898"]  )
        
        return(df)
        

In [12]:
#results("06-30-2019", 3918898)

In [13]:
app = Flask(__name__)



In [14]:
# @app.route("/")
# def index():
#     ID = request.args['input']
#     Date = request.args['date']
#     return """
#         <html><body>
#              <h1>Bank_ID {0}!</h1>
#              <h2>Date is {1}</h2>
#              Result is {2}.
#          </body></html>
#         """.format(ID, Date, results(Date, ID))
    





In [15]:
app = Flask(__name__)

@app.route('/')
 
def home():
     return """
         <html><body>
             <h2>Retrive Call report</h2><br><br>
             <form action="/ans">
                 Bank ID <input type='text' name='ID'><br><br>
                 Bank Name <select name="bank_name" disabled><br><br>
                  <option value="CIT_Bank">CIT_Bank</option>
                  <option value="DB_Bank">DB_Bank</option>
                  <option value="M_T_Bank">M_T_Bank</option>
                  <option value="NY_Com_Bank">NY_Com_Bank</option>
                  <option value="Pac_west_Bank">Pac_west_Bank</option>
                  <option value="Sign_Bank">Sign_Bank</option>
                  <option value="East_West_Bank">East_West_Bank</option>
                  <option value="OZK_Bank">OZK_Bank</option>
                </select><br><br>
                 Date <input type="date" name='Date'><br><br>
                 <input type='submit' value='Continue'>
             </form>
         </body></html>
         """



In [16]:
@app.route('/ans')
def index():
    ID = request.args['ID']
    Date = request.args['Date']
    res = results(Date, ID).to_html(header="true", table_id="table", index=False)
    #print(res)
    return """
        <html><body>
             <h1>Bank_ID {0}</h1>
             <h2>Date is {1}</h2>
             <h2>Result is </h2> 
             <h3>{2}</h3>
         </body></html>
        """.format(ID, Date,res)


In [17]:
# @app.route('/download')
# def download():
#     ID = request.args['ID']
#     Date = request.args['Date']
#     res = results(Date, ID).to_html(header="true", table_id="table", index=False)
#     resp = make_response(res.to_csv())
#     resp.headers["Content-Disposition"] = "attachment; filename=export.csv"
#     resp.headers["Content-Type"] = "text/csv"
#     return (resp)
    


In [18]:
if __name__ == "__main__":
    app.run(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Nov/2019 17:36:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Nov/2019 17:36:43] "GET /ans?ID=497404&Date=2019-10-30 HTTP/1.1" 200 -
127.0.0.1 - - [07/Nov/2019 17:37:01] "GET /ans?ID=497404&Date=2017-12-31 HTTP/1.1" 200 -
127.0.0.1 - - [07/Nov/2019 17:37:16] "GET /ans?ID=497404&Date=2017-12-31 HTTP/1.1" 200 -
127.0.0.1 - - [07/Nov/2019 17:37:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Nov/2019 17:38:55] "GET /ans?ID=497404&Date=2019-09-30 HTTP/1.1" 200 -
